### DACON_유전체 정보 품종 분류: svc

* eda 결과를 보면 피처 trait이 1인것은 모두 품종이 A이기에 각각 분류기를 따로 만들었음.
* 분류를 하기 위해서는 차이가 명확해야한다는 생각으로 카이제곱을 통해 통계량의 값이 같다고 나오는 피처를 탐색하여 제고하고자 함. 각 snp별로 가질수 있는 형질별 품종의 비율이 같은지에 대해, 각 snp별 하나의 형질에서 가질수 품종의 비율이 같은지에 대해 카이제곱을 진행한 결과 두 가지 모두 유의수준 0.01로 유의하게 나온건 snp3밖에 없었다. (이 논리가 맞는지 모르겠지만 일단 해봤다)
* 추가)
* 이상치 확인을 위해 3차원에서 사분위 수를 이용하여 이상치제거를 하였지만 성능개선에 도움이 안됐음

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
test_path = 'C:/Users/hyo/Downloads/snp_data/test.csv'
train_path = "C:/Users/hyo/Downloads/snp_data/train.csv"
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [3]:
#분류기 나누기
id_test = test['id']
A_test_id = test[test['trait']==1].index
BC_test_id = test[test['trait']!=1].index

train = train.drop(train[train['trait']==1].index)
test = test.drop(test[test['trait']==1].index)

train_len = len(train)
print(train_len)

193


In [4]:
train = train.drop(columns = ["id", "father","mother","gender","trait"])
test = test.drop(columns = ["id", "father","mother","gender","trait"])

In [5]:
dataset = pd.concat([train,test],axis=0)

In [6]:
#snp3제거
dataset = dataset.drop(columns = "SNP_03")

In [7]:
#염색체 정보별
dataset['chrom_9'] = dataset['SNP_12']+"-"+dataset['SNP_13']+"-"+dataset['SNP_14']
dataset['chrom_6'] = dataset['SNP_02']+"-"+dataset['SNP_04']+"-"+dataset['SNP_05']+"-"+dataset['SNP_06']+"-"+dataset['SNP_07']+"-"+dataset['SNP_08']+"-"+dataset['SNP_09']

In [8]:
#공통된 이름별
dataset['Bov'] = dataset['SNP_05']+"-"+dataset['SNP_06']+"-"+dataset['SNP_08']+"-"+dataset['SNP_15']
dataset['Hap'] = dataset['SNP_07']+"-"+dataset['SNP_12']+"-"+dataset['SNP_14']
dataset['BTB'] = dataset['SNP_10']+"-"+dataset['SNP_13']
dataset['ARS'] = dataset['SNP_02']+"-"+dataset['SNP_04']+"-"+dataset['SNP_09']+"-"+dataset['SNP_11']

In [9]:
# 전체 
dataset['all_snp'] =  dataset['SNP_01']+"-"+dataset['SNP_02']+"-"+dataset['SNP_04']+"-"+dataset['SNP_05']+"-"+dataset['SNP_06']+"-"+dataset['SNP_07']+"-"+ dataset['SNP_08']+"-"+dataset['SNP_09']+"-"+dataset['SNP_10']+"-"+dataset['SNP_11']+"-"+dataset['SNP_12']+"-"+dataset['SNP_13']+"-"+dataset['SNP_14']+"-"+dataset['SNP_15']

In [10]:
#원핫
dataset_ohe = pd.get_dummies(dataset,columns = dataset.columns.drop("class"),drop_first=True) #multicollinearity를 막기위한 drop_first 옵션
train_ohe = dataset_ohe[:train_len].copy()
test_ohe = dataset_ohe[train_len:].copy().drop(columns="class")

class_map = {"A":0,"B":1,"C":2}
train_ohe["class"]=train_ohe["class"].map(class_map).astype(int)
X_train_ohe = train_ohe.drop(columns = "class")
Y_train_ohe = train_ohe["class"]

In [11]:
from sklearn.svm import SVC

In [12]:
reg = SVC(C=100, kernel='linear') 
reg.fit(X_train_ohe, Y_train_ohe) 

SVC(C=100, kernel='linear')

In [13]:
results=reg.predict(test_ohe)

In [14]:
df_result = pd.DataFrame(results)
df_result['class'] = results

In [15]:
de_result = df_result.drop(columns=0)

In [16]:
A = pd.DataFrame(A_test_id)
A['class'] = 'A'
A = A.rename(columns={0:'index'})

In [17]:
BC = pd.DataFrame(BC_test_id)
BC = BC.rename(columns={0:'index'})

In [18]:
BC = pd.concat((BC,df_result), axis=1)

In [19]:
BC = BC.drop(columns=0)

In [20]:
result = pd.concat((A,BC), axis=0)
result.set_index('index', inplace=True)

In [21]:
result.sort_index(inplace=True)

In [22]:
class_map_inv = {"A":"A",1:"B",2:"C"}
result["class"] = result["class"].map(class_map_inv)

In [23]:
result['id'] = id_test

In [26]:
result = result[['id', 'class']]
result

,id,class
index,,
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C


In [27]:
result.to_csv('./svc.csv', index=False)

#### 느낀점
* 분류기는 절대로 따로 만들면 안된다. 데이터도 부족한데 더 부족해지게 만드는 거니까. 차라리 이번과 상황이 같다면 다 합쳐서 예측하고 그다음 특징이 1인것의 값을 'A'로 바꿔주도록 하자.
* 카이제곱 함부로 활용하면 안된다. 자칫하면 가중치를 잘못 두는 거기에 학습이 잘못될수 있다.(솔직히 저기서 쓴것도 논리 빈약)
* 차원의 저주 조심하자.
* 데이터 수가 적을수록 학습과 테스트 데이터 셋의 예측력 사이 변동은 커진다.(이상치도 조심해서 빼야함)
* lightgbm은 데이터수가 만개 이상 될때 추천한다.
* eda 정말 중요하다.
* 예상외로 서브정보가 많은 도움이 된다.
* 데이콘 대회는 자동으로 점수가 가장 높은게 올라가기에 마지막 제출전 같은 점수라면 최근 파일로 제출을 변경할 것.
* `툴 아는것이 중요하다`